# Data Analysis

In [32]:
%pip -q install pandas pyarrow matplotlib hypersync duckdb 

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import pyarrow.parquet as pq
pf = pq.ParquetFile("tx_sample_with_date.parquet")
tbl = pf.read()                 # safe for small files
df = tbl.to_pandas()
print("DataFrame shape:", df.shape)
display(df.head(5))

In [ ]:
import os
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

def charts(
    path: str,
    sample_fraction: float = 1.0,   
    outdir: str = ".",
    date_col: str = "date",
    from_col: str = "from",
    to_col: str = "to",
    gas_price_col: str = "gas_price", # wei
    gas_used_col: str = "gas"      
):
    os.makedirs(outdir, exist_ok=True)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")

    # 1) Estimated Daily Transactions (Million)
    q_tx = f"""
    WITH t AS (
      SELECT CAST({date_col} AS DATE) AS date
      FROM read_parquet('{path}')
    ),
    mx AS ( SELECT max(date) AS maxd FROM t )
    SELECT t.date AS date,
           COUNT(*)::DOUBLE / {sample_fraction} AS tx_count_estimated
    FROM t, mx
    WHERE t.date < mx.maxd               -- exclude last (max) date
    GROUP BY 1
    ORDER BY 1
    """
    df_tx = duckdb.sql(q_tx).df()

    plt.figure(figsize=(10, 5))
    plt.plot(pd.to_datetime(df_tx["date"]), df_tx["tx_count_estimated"]/10**6,
             linewidth=1.0, markersize=3)
    plt.title("Estimated Daily Transactions (Million)")
    plt.xlabel("Date"); plt.ylabel("Estimated Transaction Count (M)")
    plt.grid(True, alpha=0.3)
    plt.gcf().autofmt_xdate()
    plt.tight_layout()
    f_tx = os.path.join(outdir, f"estimated_daily_transactions_{ts}.png")
    plt.savefig(f_tx, dpi=160)
    plt.show()

    # 2) Cumulative Distinct Addresses (Millions), scaled by 1/sample_fraction
    q_addr = f"""
    WITH base AS (
      SELECT "{from_col}" AS address, CAST({date_col} AS DATE) AS date FROM read_parquet('{path}')
      UNION ALL
      SELECT "{to_col}"   AS address, CAST({date_col} AS DATE) AS date FROM read_parquet('{path}')
    ),
    mx AS ( SELECT max(date) AS maxd FROM base ),
    flt AS (
      SELECT address, date FROM base, mx
      WHERE date < mx.maxd
    ),
    first_seen AS (
      SELECT address, MIN(date) AS first_date
      FROM flt
      GROUP BY address
    ),
    daily_new AS (
      SELECT first_date AS date, COUNT(*) AS new_addresses
      FROM first_seen
      GROUP BY 1
    ),
    all_days AS (
      SELECT DISTINCT date FROM flt
    )
    SELECT
      d.date,
      (SUM(COALESCE(n.new_addresses, 0)) OVER (ORDER BY d.date)) / {sample_fraction} AS cum_addresses
    FROM all_days d
    LEFT JOIN daily_new n ON n.date = d.date
    ORDER BY d.date
    """
    df_addr = duckdb.sql(q_addr).df()

    plt.figure(figsize=(10, 5))
    plt.plot(pd.to_datetime(df_addr["date"]), df_addr["cum_addresses"]/1_000_000,
             linewidth=1.0, markersize=3)
    plt.title("Cumulative Distinct Addresses")
    plt.xlabel("Date"); plt.ylabel("Total Distinct Addresses (millions)")
    plt.grid(True, alpha=0.3)
    plt.gcf().autofmt_xdate(); plt.tight_layout()
    f_addr = os.path.join(outdir, f"cumulative_distinct_addresses_{ts}.png")
    plt.savefig(f_addr, dpi=160)
    plt.show()

    # 3) Average Gas Price per Day (Gwei)
    q_gprice = f"""
    WITH t AS (
      SELECT
        CAST({date_col} AS DATE)                AS date,
        TRY_CAST({gas_price_col} AS DOUBLE)     AS gas_price
      FROM read_parquet('{path}')
    ),
    mx AS (SELECT max(date) AS maxd FROM t),
    flt AS (
      SELECT date, gas_price
      FROM t, mx
      WHERE date < mx.maxd
        AND gas_price IS NOT NULL
    )
    SELECT
      date,
      AVG(gas_price) / 1e9 AS avg_gas_price_gwei
    FROM flt
    GROUP BY 1
    ORDER BY 1
    """
    df_gprice = duckdb.sql(q_gprice).df()

    plt.figure(figsize=(10,5))
    plt.plot(pd.to_datetime(df_gprice["date"]), df_gprice["avg_gas_price_gwei"],
             linewidth=1.0, markersize=3)
    plt.title("Average Gas Price per Day (Gwei)")
    plt.xlabel("Date"); plt.ylabel("Avg Gas Price (Gwei)")
    plt.grid(True, alpha=0.3)
    plt.gcf().autofmt_xdate(); plt.tight_layout()
    f_gprice = os.path.join(outdir, f"avg_gas_price_gwei_{ts}.png")
    plt.savefig(f_gprice, dpi=160)
    plt.show()

    # 4) Daily Total Gas Used (Billion)
    q_gused = f"""
    WITH tx AS (
      SELECT
        CAST({date_col} AS DATE)                AS date,
        TRY_CAST({gas_used_col} AS DOUBLE)      AS gas_used
      FROM read_parquet('{path}')
    ),
    last_day AS (SELECT max(date) AS max_date FROM tx),
    tx_clean AS (
      SELECT date, gas_used
      FROM tx, last_day
      WHERE date < last_day.max_date
        AND gas_used IS NOT NULL
    )
    SELECT
      date,
      SUM(gas_used) AS total_gas_used_sample
    FROM tx_clean
    GROUP BY 1
    ORDER BY 1
    """
    df_gused = duckdb.sql(q_gused).df()

    plt.figure(figsize=(10,5))
    plt.plot(pd.to_datetime(df_gused["date"]),
             df_gused["total_gas_used_sample"]/10**9,
             linewidth=1.0, markersize=3)
    plt.title("Daily Total Gas Used")
    plt.xlabel("Date"); plt.ylabel("Total Gas Used (Billion)")
    plt.grid(True, alpha=0.3)
    plt.gcf().autofmt_xdate(); plt.tight_layout()
    f_gused = os.path.join(outdir, f"daily_total_gas_used_{ts}.png")
    plt.savefig(f_gused, dpi=160)
    plt.show()

    return {
        "tx_png": f_tx,
        "addr_png": f_addr,
        "gprice_png": f_gprice,
        "gused_png": f_gused,
    }

In [ ]:
files = charts(
    path="tx_sample_with_date.parquet",
    sample_fraction=0.00168,  
    outdir="./charts",
    date_col="date",
    from_col="from",
    to_col="to",
    gas_price_col="gas_price",
    gas_used_col="gas"       
)
print(files)
files = charts(
    path="optimism_sample_with_date.parquet",
    sample_fraction=0.0076,  
    outdir="./charts",
    date_col="date",
    from_col="from",
    to_col="to",
    gas_price_col="gas_price",
    gas_used_col="gas"       
)
print(files)